In [2]:
import gezi
from gezi import tqdm
import json

In [3]:
news_names = ['did', 'cat', 'sub_cat', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities']

In [4]:
files = [
  '../input/train/news.tsv',
  '../input/dev/news.tsv',
  '../input/test/news.tsv',
]

In [5]:
cats = gezi.WordCounter()
sub_cats = gezi.WordCounter()
entities = gezi.WordCounter()
entity_types = gezi.WordCounter()

In [6]:
TEST_START = 1000000

did_vocab = gezi.Vocab('../input/did.txt')

def get_did_count(did):
  count = did_vocab.count_key(did)
  if count >= TEST_START:
    count = count - TEST_START + 1
  return count

dids = set()
for file in files:
  total = len(open(file).readlines())
  for line in tqdm(open(file), total=total):
    l = line.strip().split('\t')
    did, cat, sub_cat, title_entities, abstract_entities = l[0], l[1], l[2], l[-2], l[-1]
    if did in dids:
      continue
    dids.add(did)
    count = 1 if 'test' in file or cat in cats.counter else TEST_START 
    cats.add(cat, count)
    count = 1 if 'test' in file or sub_cat in sub_cats.counter else TEST_START 
    sub_cats.add(sub_cat, count)
    title_entities = json.loads(title_entities)
    abstract_entities = json.loads(abstract_entities)
    
    did_count = get_did_count(did)
    for m in title_entities:
      count = 0 if 'test' in file or m['WikidataId'] in entities.counter else TEST_START
      count += did_count
      entities.add(m['WikidataId'], count)
      count = 0 if 'test' in file or m['Type'] in entity_types.counter else TEST_START 
      count += did_count
      entity_types.add(m['Type'], count)
    for m in abstract_entities:
      count = 0 if 'test' in file or m['WikidataId'] in entities.counter else TEST_START 
      count += did_count
      entities.add(m['WikidataId'], count)
      count = 0 if 'test' in file or m['Type'] in entity_types.counter else TEST_START 
      count += did_count
      entity_types.add(m['Type'], count)

100%|██████████| 120961/120961 [00:01<00:00, 90745.61it/s]


In [7]:
len(cats.counter)

18

In [8]:
len(sub_cats.counter)

293

In [9]:
len(entities.counter)

50925

In [10]:
len(entity_types.counter)

22

In [11]:
entities.counter

Counter({'Q80976': 1168396,
         'Q43274': 2169423,
         'Q9682': 1578646,
         'Q2796': 1215868,
         'Q483551': 4572446,
         'Q312': 1747567,
         'Q193583': 1205711,
         'Q622899': 1029987,
         'Q212': 6785599,
         'Q155223': 3123281,
         'Q3179593': 1062288,
         'Q171171': 1141364,
         'Q371820': 2939415,
         'Q16555': 1583189,
         'Q1215884': 12856335,
         'Q1406': 1044806,
         'Q94': 1034053,
         'Q49233': 1297020,
         'Q298': 1159474,
         'Q2887': 1013540,
         'Q1323662': 1001920,
         'Q19809': 3400605,
         'Q7957644': 1006370,
         'Q3191485': 1015569,
         'Q219714': 1659450,
         'Q984336': 1029628,
         'Q205033': 1753853,
         'Q1211586': 1054477,
         'Q173476': 1002905,
         'Q3403879': 1052365,
         'Q209330': 2809219,
         'Q1166': 3179834,
         'Q10479': 3733645,
         'Q466786': 1121419,
         'Q910928': 1180967,
      

In [12]:
cats.save('../input/cat.txt')
sub_cats.save('../input/sub_cat.txt')
entities.save('../input/entity.txt')
entity_types.save('../input/entity_type.txt')

total_count 18130361 unknown_count 0 total_word 18
total_count 285130094 unknown_count 0 total_word 293
total_count 45063435317 unknown_count 0 total_word 50925
total_count 1008435317 unknown_count 0 total_word 22


In [13]:
!mkdir -p ../input/news

In [14]:
# 获取整体全量news.tsv
dids = set()
with open('../input/news/news.tsv', 'w') as out:
  for file in files:
    total = len(open(file).readlines())
    for line in tqdm(open(file), total=total):
      l = line.strip().split('\t')  
      did = l[0]
      if did in dids:
        continue
      dids.add(did)
      print(line, file=out, end='')

100%|██████████| 120961/120961 [00:00<00:00, 228595.74it/s]
